# Testing Difference in Means
- Using `statsmodels.power.tt_ind_solve_power`
- https://www.statsmodels.org/dev/generated/statsmodels.stats.power.tt_ind_solve_power.html#statsmodels.stats.power.tt_ind_solve_power
- solve for any one parameter of the power of a two sample t-test

## Terminology
In `tt_ind_solve_power`, `effect_size` is the standardized effect size, also known as Cohen's d.

In [4]:
import numpy as np
import pandas
from statsmodels.stats import power as sm_power

In [5]:
m1 = 30  # Mean of group 1
m2 = 28  # Mean of group 2
s1 = 3  # Standard deviation of group 1
s2 = 2  # Standard deviation of group 2
n1 = 20  # Sample size of group 1
n2 = 25  # Sample size of group 2
alpha = 0.05
power = 0.85
ratio = 2/3

In [6]:
# Function to calculate pooled standard deviation
def pooled_std(s1, s2, n1, n2):
    return np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))

# statsmodels calls this the standardized effect size
# Function to calculate Cohen's d
def cohens_d(m1, m2, s1, s2, n1, n2):
    # Calculate the pooled standard deviation
    pooled_sd = pooled_std(s1, s2, n1, n2)
    
    # Calculate Cohen's d
    d = (m1 - m2) / pooled_sd
    return d

In [7]:
nobs_group_1 = sm_power.tt_ind_solve_power(
    effect_size= cohens_d(m1, m2, s1, s2, n1, n2),
    alpha=alpha,
    power=power,
    ratio= ratio
)
nobs_group_1

36.0342628397395

In [8]:
nobs_group_2 = nobs_group_1 * ratio
nobs_group_2

24.022841893159665

In [9]:
total_nobs= nobs_group_1 + nobs_group_2
total_nobs

60.05710473289916

We want a function to test differences in mean similar to R's AB_t2n package.

We create an adapter around stasmodels.stats.power.tt_ind_solver_power to solve for the number of observations (nobs).

The wrapper's parameters are:
- diff_means - difference in means
- std1 - standard deviation of group 1
- std2 - standard deviation of group 2
- alpha = 0.05 - significance level, default to 0.05
- power - desired power
- ratio - (n2/n1) as defined by tt_ind_solver_power

The wrapper functions to:
- Determines starting values for n1 and n2 to pass to the solver. The size of n1 and n2 are based on the ratio, such that n2 = 30 and n2/n1 = ratio or n1 = 30/ratio.
- Determines m1 and m2 where m1=diff_means and m2 = 0. 
- Passes remaining values on to the solver.

In [10]:
def AB_nobs(diff_means, std1, std2, power, ratio = 1, alpha = 0.05):
    # n1_start and n2_start are starting values passed to the solver
    nobs2_start = 30
    nobs1_start = nobs2_start/ratio
    m1 = diff_means
    m2 = 0
    nobs1 = sm_power.tt_ind_solve_power(
        effect_size= cohens_d(m1, m2, std1, std2, nobs1_start, nobs2_start),
        nobs1 = None,
        alpha=alpha,
        power = power,
        ratio = ratio
    )
    # ratio as defined by stats.power.tt_ind_solve_power
    nobs2 = nobs1 * ratio
    nobs_total = nobs1 + nobs2
    return pandas.Series({"nobs1": nobs1, "nobs2": nobs2, "nobs_total": nobs_total})

In [11]:
AB_nobs(2, 2, 3, 0.85, 2/3)

nobs1         34.784308
nobs2         23.189539
nobs_total    57.973847
dtype: float64

In [12]:
def AB_power(diff_means, nobs1, nobs2, std1, std2, alpha= 0.05):
    # ratio as defined by stats.power.tt_ind_solve_power
    ratio = nobs2/nobs1
    m1 = diff_means
    m2 = 0
    return sm_power.tt_ind_solve_power(
        effect_size= cohens_d(m1, m2, std1, std2, nobs1, nobs2),
        nobs1 = nobs1,
        alpha=alpha,
        ratio = ratio
    )


In [13]:
AB_power(2, 34.784308, 23.189539, 2, 3)

np.float64(0.8502430693080085)